# Classification Notebook

This notebook is an exact copy of the example provided by **DeepNetts** at https://github.com/deepnetts/deepnetts-communityedition/blob/community-visrec/deepnetts-examples/src/main/java/deepnetts/examples/IrisFlowersClassifier.java

We need to load the `deepnetts-core` jar and other dependencies (via Maven pom invocations) before we get started. The deepnetts-examples-n.nn.jar (for e.g. deepnetts-examples-1.11.jar) is a result of building the `deepnetts-communityedition` project and copying the `jar` from the `target` folder into the `notebooks` folder.

In [1]:
%%loadFromPOM

<properties>
    <deepnetts.version>1.12</deepnetts.version>
    <visrec.version>1.0.1</visrec.version>
</properties>

<dependency>
  <groupId>javax.visrec</groupId>
  <artifactId>visrec-api</artifactId>
  <version>1.0.1</version>
</dependency>
<dependency>
  <groupId>javax.visrec</groupId>
  <artifactId>visrec-ri</artifactId>
  <version>1.0.1</version>
  <type>jar</type>
  <exclusions>
      <exclusion>
          <groupId>com.deepnetts</groupId>
          <artifactId>deepnetts-core</artifactId>
      </exclusion>
  </exclusions>
</dependency>
<dependency>
  <groupId>org.knowm.xchart</groupId>
  <artifactId>xchart</artifactId>
  <version>3.5.2</version>
</dependency>
<dependency>
    <groupId>com.deepnetts</groupId>
    <artifactId>deepnetts-core</artifactId>
    <version>${deepnetts.version}</version>
</dependency> 

In [2]:
%%loadFromPOM

<!--log4j2 dependencies -->
<properties>
    <log4j2.version>2.7</log4j2.version>
</properties>
<dependency>
    <groupId>org.apache.logging.log4j</groupId>
    <artifactId>log4j-api</artifactId>
    <version>${log4j2.version}</version>
</dependency>
<dependency>
    <groupId>org.apache.logging.log4j</groupId>
    <artifactId>log4j-core</artifactId>
    <version>${log4j2.version}</version>
</dependency>
<dependency>
    <groupId>org.apache.logging.log4j</groupId>
    <artifactId>log4j-1.2-api</artifactId>
    <version>${log4j2.version}</version>
</dependency>
<dependency>
    <groupId>org.apache.logging.log4j</groupId>
    <artifactId>log4j-slf4j-impl</artifactId>
    <version>${log4j2.version}</version>
</dependency>
<dependency>
    <groupId>org.apache.logging.log4j</groupId>
    <artifactId>log4j-jcl</artifactId>
    <version>${log4j2.version}</version>
</dependency>

In [3]:
import java.io.IOException;

In [4]:
import deepnetts.data.DataSets;
import deepnetts.eval.ClassifierEvaluator;
import deepnetts.eval.ConfusionMatrix;
import deepnetts.net.FeedForwardNetwork;
import deepnetts.net.layers.activation.ActivationType;
import deepnetts.net.loss.LossType;
import deepnetts.net.train.BackpropagationTrainer;
import deepnetts.net.train.opt.OptimizerType;
import deepnetts.util.DeepNettsException;
import javax.visrec.ml.data.DataSet;
import javax.visrec.ml.eval.EvaluationMetrics;

In [5]:
int numInputs = 4;  // corresponds to number of input features
int numOutputs = 3; // corresponds to number of possible classes/categories
double test_train_ratio = 0.6;

These imports are for the provenance system, which we'll get to in a minute.

## Loading the data

In [6]:
System.out.println("~~ Running DeepNetts Machine Classification");
System.out.println("~~~ Loading the data");
DataSet dataSet = DataSets.readCsv("../deepnetts-examples/datasets/iris_data_normalised.txt", numInputs, numOutputs, true);

~~ Running DeepNetts Machine Classification
~~~ Loading the data


## Splitting datasets

In [7]:
System.out.println("~~~~~~ Splitting datasets");
DataSet[] trainTestSet = dataSet.split(test_train_ratio, 1 - test_train_ratio);
System.out.println(String.format("Training data size = %d rows", trainTestSet[0].size()));
System.out.println(String.format("Testing data size = %d rows", trainTestSet[1].size()));

~~~~~~ Splitting datasets
Training data size = 90 rows
Testing data size = 60 rows


## Training the model

In [8]:
System.out.println("~~~ Training the model ~~~");
// create instance of multi addLayer percetpron using builder
FeedForwardNetwork neuralNet = FeedForwardNetwork.builder()
        .addInputLayer(numInputs)
        .addFullyConnectedLayer(5, ActivationType.TANH)
        .addOutputLayer(numOutputs, ActivationType.SOFTMAX)
        .lossFunction(LossType.CROSS_ENTROPY)
        .randomSeed(456)
        .build();
System.out.println(neuralNet.toString());

~~~ Training the model ~~~
Input Layer { width:4, height:1, depth:1 }
Fully Connected Layer { width:5 activation:TANH}
Output Layer { width:3, activation:SOFTMAX}



In [9]:
BackpropagationTrainer trainer = neuralNet.getTrainer();
trainer.setMaxError(0.04f);
trainer.setLearningRate(0.01f);
trainer.setMomentum(0.9f);
trainer.setOptimizer(OptimizerType.MOMENTUM);

neuralNet.train(trainTestSet[0]);
System.out.println("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~");
System.out.println("");

------------------------------------------------------------------------------------------------------------------------------------------------
TRAINING NEURAL NETWORK
------------------------------------------------------------------------------------------------------------------------------------------------
Epoch:1, Time:56ms, TrainError:0.8182211, TrainErrorChange:0.8182211, TrainAccuracy: 0.5833333
Epoch:2, Time:10ms, TrainError:0.4891482, TrainErrorChange:-0.3290729, TrainAccuracy: 0.6388889
Epoch:3, Time:8ms, TrainError:0.38582256, TrainErrorChange:-0.103325635, TrainAccuracy: 0.943922
Epoch:4, Time:14ms, TrainError:0.32299483, TrainErrorChange:-0.062827736, TrainAccuracy: 0.9575758
Epoch:5, Time:16ms, TrainError:0.2768873, TrainErrorChange:-0.04610753, TrainAccuracy: 0.9575758
Epoch:6, Time:4ms, TrainError:0.24381275, TrainErrorChange:-0.033074543, TrainAccuracy: 0.9575758
Epoch:7, Time:1ms, TrainError:0.21998572, TrainErrorChange:-0.023827031, TrainAccuracy: 0.9575758
Epoch:

Epoch:80, Time:1ms, TrainError:0.07204541, TrainErrorChange:-7.935688E-4, TrainAccuracy: 0.97840476
Epoch:81, Time:0ms, TrainError:0.071276836, TrainErrorChange:-7.685721E-4, TrainAccuracy: 0.97840476
Epoch:82, Time:1ms, TrainError:0.07053256, TrainErrorChange:-7.4427575E-4, TrainAccuracy: 0.97840476
Epoch:83, Time:1ms, TrainError:0.06981139, TrainErrorChange:-7.211715E-4, TrainAccuracy: 0.97840476
Epoch:84, Time:1ms, TrainError:0.06911232, TrainErrorChange:-6.990656E-4, TrainAccuracy: 0.97840476
Epoch:85, Time:0ms, TrainError:0.06843481, TrainErrorChange:-6.775111E-4, TrainAccuracy: 0.97840476
Epoch:86, Time:1ms, TrainError:0.067778215, TrainErrorChange:-6.565973E-4, TrainAccuracy: 0.97840476
Epoch:87, Time:1ms, TrainError:0.06714154, TrainErrorChange:-6.3667446E-4, TrainAccuracy: 0.97840476
Epoch:88, Time:0ms, TrainError:0.06652436, TrainErrorChange:-6.1718374E-4, TrainAccuracy: 0.97840476
Epoch:89, Time:0ms, TrainError:0.06592583, TrainErrorChange:-5.985275E-4, TrainAccuracy: 0.9784

Epoch:161, Time:0ms, TrainError:0.046603896, TrainErrorChange:-1.2154877E-4, TrainAccuracy: 0.97840476
Epoch:162, Time:0ms, TrainError:0.04648411, TrainErrorChange:-1.1978671E-4, TrainAccuracy: 0.97840476
Epoch:163, Time:0ms, TrainError:0.046366148, TrainErrorChange:-1.1796132E-4, TrainAccuracy: 0.97840476
Epoch:164, Time:0ms, TrainError:0.046249814, TrainErrorChange:-1.16333365E-4, TrainAccuracy: 0.97840476
Epoch:165, Time:1ms, TrainError:0.04613515, TrainErrorChange:-1.14664435E-4, TrainAccuracy: 0.97840476
Epoch:166, Time:0ms, TrainError:0.04602216, TrainErrorChange:-1.12988055E-4, TrainAccuracy: 0.97840476
Epoch:167, Time:1ms, TrainError:0.04591072, TrainErrorChange:-1.1144206E-4, TrainAccuracy: 0.97840476
Epoch:168, Time:1ms, TrainError:0.04580085, TrainErrorChange:-1.0986999E-4, TrainAccuracy: 0.97840476
Epoch:169, Time:0ms, TrainError:0.045692477, TrainErrorChange:-1.0837242E-4, TrainAccuracy: 0.97840476
Epoch:170, Time:1ms, TrainError:0.04558542, TrainErrorChange:-1.0705739E-4,

Epoch:242, Time:1ms, TrainError:0.04001064, TrainErrorChange:-6.0688704E-5, TrainAccuracy: 0.97840476
Epoch:243, Time:1ms, TrainError:0.039950173, TrainErrorChange:-6.0465187E-5, TrainAccuracy: 0.97840476

TRAINING COMPLETED
Total Training Time: 2851ms
------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



## Evaluating the model

In [10]:
System.out.println("~~~ Evaluating the model ~~~");
ClassifierEvaluator evaluator = new ClassifierEvaluator();
EvaluationMetrics em = evaluator.evaluate(neuralNet, trainTestSet[1]);

~~~ Evaluating the model ~~~


In [11]:
System.out.println("CLASSIFIER EVALUATION METRICS");
System.out.println(em);
System.out.println("CONFUSION MATRIX");
ConfusionMatrix cm = evaluator.getConfusionMatrix();
System.out.println(cm);

CLASSIFIER EVALUATION METRICS
Accuracy: 0.93460923 (How often is classifier correct in total)
Precision: 0.96491224 (How often is classifier correct when it gives positive prediction)
F1Score: 0.96491224 (Average of precision and recall)
Recall: 0.96491224 (When it is actually positive class, how often does it give positive prediction)

CONFUSION MATRIX
                  none     setosa versicolor  virginica
       none          0          0          0          0
     setosa          0         21          0          0
 versicolor          0          0         20          2
  virginica          0          0          0         17



## Credits & References

#### A good place to learn more about Classifications, the associated metrics and other concepts is the Tutorial on Tribuo's git repo, see https://github.com/oracle/tribuo/blob/main/tutorials/irises-tribuo-v4.ipynb., you can find more explanations and further reading material there.